<a href="https://colab.research.google.com/github/pankaj7322/chatbot-using-RAG-model/blob/main/Retrieval_Augmented_Generation_(RAG)_Model_for_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Upload important packages

In [ ]:
!pip install faiss-gpu
!pip install sentence-transformers
!pip install qdrant-client
!python -m pip install cohere --upgrade

  Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.2

## Import the necessary Packages

In [ ]:

from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np
import cohere

In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://d592a8e8-bf4f-42c2-8733-49bfdb35c394.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="d9oXnGCA1wBjcjUF977pT7rjAyBKh3-zgxOO2g95CQafCM9LAvf6VA",
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='health_faqs')]


- for implementation of the chatbot I uses the 100 Question and answer Related to the Healthcare

In [ ]:

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example texts to generate embeddings for
common_questions = [
    {"question": "What are the symptoms of diabetes?", "answer": "Common symptoms include increased thirst, frequent urination, and fatigue."},
    {"question": "How often should I get a check-up?", "answer": "It's generally recommended to have an annual check-up."},
     {"question": "How often should I get a check-up?", "answer": "It's generally recommended to have an annual check-up."},
    {"question": "What are the signs of a heart attack?", "answer": "Common signs include chest pain, shortness of breath, and discomfort in other areas of the upper body."},
    {"question": "What should I do if I'm feeling anxious?", "answer": "Consider practicing deep breathing exercises, talking to a friend, or consulting a mental health professional."},
    {"question": "What vaccinations are recommended for adults?", "answer": "Adults should consider vaccinations like the flu shot, Tdap, and shingles vaccine."},
    {"question": "What are some healthy snack options?", "answer": "Healthy snacks include fruits, nuts, yogurt, and whole-grain crackers."},
    {"question": "How can I manage my weight effectively?", "answer": "Focus on a balanced diet, regular exercise, and consult a healthcare provider for personalized advice."},
    {"question": "What is a balanced diet?", "answer": "A balanced diet includes a variety of foods: fruits, vegetables, whole grains, protein, and healthy fats."},
    {"question": "How much water should I drink daily?", "answer": "It's generally recommended to drink about 8 cups (2 liters) of water per day, but needs can vary."},
    {"question": "What are some signs of dehydration?", "answer": "Signs include thirst, dry mouth, fatigue, and dark-colored urine."},
    {"question": "How do I know if I have allergies?", "answer": "Common symptoms include sneezing, itching, and rashes. A healthcare provider can perform tests for confirmation."},
    {"question": "What should I do if I have a fever?", "answer": "Stay hydrated, rest, and consider over-the-counter medications. If the fever persists, consult a doctor."},
    {"question": "What are the benefits of regular exercise?", "answer": "Regular exercise can improve cardiovascular health, strengthen muscles, and enhance mental well-being."},
    {"question": "How can I improve my sleep quality?", "answer": "Establish a regular sleep schedule, create a restful environment, and avoid screens before bedtime."},
    {"question": "What is hypertension?", "answer": "Hypertension, or high blood pressure, is a condition that can lead to serious health problems if not managed."},
    {"question": "How can I reduce stress?", "answer": "Consider techniques like meditation, yoga, deep breathing, and regular physical activity."},
    {"question": "When should I seek medical attention?", "answer": "Seek medical attention if you experience severe symptoms, such as difficulty breathing, chest pain, or sudden confusion."},
    {"question": "What are some common side effects of medications?", "answer": "Common side effects can include nausea, dizziness, and fatigue, but can vary by medication."},
    {"question": "What should I do if I feel faint?", "answer": "Sit or lie down, stay hydrated, and avoid sudden movements. If it persists, seek medical advice."},
    {"question": "What are the symptoms of depression?", "answer": "Symptoms include persistent sadness, loss of interest in activities, and changes in appetite or sleep."},
    {"question": "What is the best way to quit smoking?", "answer": "Consider seeking support through counseling, medications, and support groups."},
    {"question": "What is a heart-healthy diet?", "answer": "A heart-healthy diet includes fruits, vegetables, whole grains, lean proteins, and healthy fats."},
    {"question": "How often should I exercise?", "answer": "Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week."},
    {"question": "What are the signs of an allergic reaction?", "answer": "Signs may include hives, swelling, difficulty breathing, and stomach pain."},
    {"question": "How can I manage my cholesterol levels?", "answer": "Eat a heart-healthy diet, exercise regularly, and follow your doctor's recommendations."},
    {"question": "What should I do if I have a cold?", "answer": "Rest, stay hydrated, and consider over-the-counter remedies for relief."},
    {"question": "What is a panic attack?", "answer": "A panic attack is a sudden episode of intense fear or discomfort, often accompanied by physical symptoms."},
    {"question": "What are the symptoms of a stroke?", "answer": "Symptoms may include sudden numbness, confusion, trouble speaking, and difficulty walking."},
    {"question": "How do I manage arthritis pain?", "answer": "Consider medications, physical therapy, and regular low-impact exercise."},
    {"question": "What is sleep apnea?", "answer": "Sleep apnea is a condition where breathing repeatedly stops and starts during sleep."},
    {"question": "How can I prevent heart disease?", "answer": "Maintain a healthy diet, exercise regularly, avoid smoking, and manage stress."},
    {"question": "What is the difference between a cold and the flu?", "answer": "The flu typically has more severe symptoms and comes on suddenly, while a cold is milder and develops gradually."},
    {"question": "What is a healthy body weight?", "answer": "A healthy weight varies based on factors like height, age, and muscle mass. A healthcare provider can help determine what's best for you."},
    {"question": "How can I improve my mental health?", "answer": "Consider regular exercise, a balanced diet, talking to a friend, or consulting a mental health professional."},
    {"question": "What are the risks of high blood pressure?", "answer": "High blood pressure can lead to heart disease, stroke, and kidney problems."},
    {"question": "What should I eat for breakfast?", "answer": "A healthy breakfast could include oatmeal, yogurt with fruit, or a smoothie with greens."},
    {"question": "What are the signs of dehydration?", "answer": "Signs include thirst, dry mouth, fatigue, and dark-colored urine."},
    {"question": "How can I manage my diabetes?", "answer": "Monitor blood sugar levels, eat a balanced diet, exercise, and follow your doctor's recommendations."},
    {"question": "What are some common mental health disorders?", "answer": "Common disorders include anxiety, depression, bipolar disorder, and schizophrenia."},
    {"question": "What should I do if I experience a migraine?", "answer": "Rest in a quiet, dark room and consider over-the-counter medications. If severe, consult a doctor."},
    {"question": "What is the best way to stay fit?", "answer": "Incorporate a mix of cardiovascular, strength, and flexibility exercises into your routine."},
    {"question": "What are the risks of smoking?", "answer": "Smoking increases the risk of lung cancer, heart disease, stroke, and respiratory issues."},
    {"question": "How can I tell if I'm gaining weight?", "answer": "Regularly check your weight, monitor how your clothes fit, and observe changes in your body."},
    {"question": "What should I do if I have a rash?", "answer": "Avoid scratching, keep the area clean, and consult a healthcare provider if it worsens."},
    {"question": "What is the purpose of a Pap smear?", "answer": "A Pap smear screens for cervical cancer and other abnormalities."},
    {"question": "How do I recognize the signs of an infection?", "answer": "Signs may include redness, swelling, pain, and fever."},
    {"question": "What is the best way to protect my skin from the sun?", "answer": "Use sunscreen, wear protective clothing, and avoid tanning beds."},
    {"question": "What are the benefits of mindfulness?", "answer": "Mindfulness can reduce stress, improve focus, and enhance emotional well-being."},
    {"question": "How do I find a healthcare provider?", "answer": "You can ask for recommendations from friends, search online directories, or check with your insurance provider."},
    {"question": "What is the importance of mental health?", "answer": "Mental health is crucial for overall well-being and affects how we think, feel, and act."},
    {"question": "What are some tips for healthy eating?", "answer": "Eat a variety of foods, focus on whole grains, and limit processed foods and added sugars."},
    {"question": "How can I improve my gut health?", "answer": "Eat a balanced diet rich in fiber, probiotics, and stay hydrated."},
    {"question": "What are the signs of anxiety?", "answer": "Symptoms include excessive worry, restlessness, and physical symptoms like rapid heartbeat."},
    {"question": "What should I do if I suspect I have COVID-19?", "answer": "Get tested, isolate yourself, and follow public health guidelines."},
    {"question": "How can I cope with grief?", "answer": "Consider talking to a friend or therapist, joining a support group, or allowing yourself time to heal."},
    {"question": "What is the best way to boost my immune system?", "answer": "Eat a balanced diet, exercise regularly, get enough sleep, and manage stress."},
    {"question": "What are the signs of a urinary tract infection (UTI)?", "answer": "Signs may include a burning sensation during urination, frequent urges to urinate, and cloudy urine."},
    {"question": "How do I care for my mental health during stressful times?", "answer": "Practice self-care, reach out to loved ones, and consider professional support if needed."},
    {"question": "What are some effective ways to manage chronic pain?", "answer": "Consider medication, physical therapy, and stress management techniques."},
    {"question": "What is the role of a dietitian?", "answer": "A dietitian provides personalized nutrition advice and helps manage dietary needs."},
    {"question": "What are the risks of obesity?", "answer": "Obesity increases the risk of diabetes, heart disease, and certain cancers."},
    {"question": "What is the importance of sleep?", "answer": "Sleep is vital for physical health, cognitive function, and emotional well-being."},
    {"question": "How can I create a healthier home environment?", "answer": "Focus on clean air, healthy food options, and a clutter-free space."},
    {"question": "What are the benefits of regular health screenings?", "answer": "Screenings can detect health issues early, leading to better outcomes."},
    {"question": "What should I do if I feel overwhelmed?", "answer": "Take a break, practice relaxation techniques, and reach out for support."},
    {"question": "What are some tips for managing hypertension?", "answer": "Limit salt intake, exercise regularly, and follow your healthcare provider's recommendations."},
    {"question": "What is the purpose of a cholesterol test?", "answer": "A cholesterol test assesses your risk of heart disease and helps manage overall health."},
    {"question": "How do I recognize the signs of burnout?", "answer": "Signs include chronic fatigue, irritability, and decreased performance at work."},
    {"question": "What is an allergy test?", "answer": "An allergy test identifies specific allergens that may cause reactions."},
    {"question": "How can I maintain a healthy work-life balance?", "answer": "Set boundaries, prioritize tasks, and make time for self-care."},
    {"question": "What is the significance of mental health days?", "answer": "Mental health days allow individuals to recharge and prevent burnout."},
    {"question": "What should I do if I have a cut?", "answer": "Clean the cut, apply an antibiotic ointment, and cover it with a bandage."},
    {"question": "What are the symptoms of thyroid problems?", "answer": "Symptoms can include fatigue, weight changes, and changes in mood or energy levels."},
    {"question": "How do I talk to my doctor about my concerns?", "answer": "Be open, ask questions, and express your feelings clearly."},
    {"question": "What is a health savings account (HSA)?", "answer": "An HSA is a tax-advantaged account that can be used to pay for eligible medical expenses."},
    {"question": "What are the signs of pneumonia?", "answer": "Signs may include cough, fever, shortness of breath, and chest pain."},
    {"question": "How can I improve my respiratory health?", "answer": "Avoid smoking, stay active, and keep your environment clean."},
    {"question": "What should I know about prescription medications?", "answer": "Always follow your doctor's instructions, and discuss any concerns or side effects."},
    {"question": "What are some common treatments for allergies?", "answer": "Treatments include antihistamines, decongestants, and avoiding allergens."},
    {"question": "What is the role of physical therapy?", "answer": "Physical therapy helps improve movement and manage pain through exercises and treatments."},
    {"question": "What are the benefits of staying socially active?", "answer": "Staying socially active can reduce feelings of loneliness and improve mental well-being."},
    {"question": "How do I handle food cravings?", "answer": "Identify triggers, choose healthy alternatives, and practice moderation."},
    {"question": "What should I do if I experience a severe headache?", "answer": "Rest in a dark room, stay hydrated, and consult a doctor if it persists."},
    {"question": "What is the importance of hand hygiene?", "answer": "Good hand hygiene prevents the spread of infections and illnesses."},
    {"question": "What are the risks of excessive alcohol consumption?", "answer": "Excessive drinking can lead to liver disease, addiction, and increased risk of accidents."},
    {"question": "How can I promote healthy aging?", "answer": "Stay active, eat a balanced diet, stay socially connected, and manage health conditions."},
    {"question": "What is the role of a pharmacist?", "answer": "Pharmacists dispense medications, provide information, and ensure safe medication use."},
    {"question": "What should I do if I suspect food poisoning?", "answer": "Stay hydrated, rest, and seek medical attention if symptoms are severe."},
    {"question": "How can I improve my posture?", "answer": "Practice good ergonomics, strengthen core muscles, and take breaks to stretch."},
    {"question": "What are the signs of a mental health crisis?", "answer": "Signs may include severe mood changes, withdrawal, and risk-taking behavior."},
    {"question": "What is the importance of regular dental check-ups?", "answer": "Regular dental check-ups can prevent cavities, gum disease, and other oral health issues."},
    {"question": "How can I support a loved one with a mental health issue?", "answer": "Listen, offer support, and encourage them to seek professional help if needed."},
    {"question": "What are some tips for managing stress at work?", "answer": "Prioritize tasks, take breaks, and practice relaxation techniques."},
    {"question": "How do I prepare for a doctor's appointment?", "answer": "Bring a list of questions, update your medical history, and be ready to discuss your symptoms."},
    {"question": "What are the risks of not getting vaccinated?", "answer": "Not getting vaccinated can increase your risk of preventable diseases and outbreaks."},
    {"question": "How can I maintain a healthy heart?", "answer": "Eat a balanced diet, exercise regularly, manage stress, and avoid smoking."},
    {"question": "What is the purpose of a physical exam?", "answer": "A physical exam assesses your overall health and can detect potential issues early."}

]

faqs = [
    {"question": "What should I do in case of an emergency?", "answer": "Call emergency services immediately or go to the nearest emergency room."},
    {"question": "How can I improve my mental health?", "answer": "Regular exercise, a balanced diet, and talking to a mental health professional can help."},
     {"question": "What should I do in case of an emergency?", "answer": "Call emergency services immediately or go to the nearest emergency room."},
    {"question": "How can I improve my mental health?", "answer": "Regular exercise, a balanced diet, and talking to a mental health professional can help."},
    {"question": "What is the difference between a cold and the flu?", "answer": "The flu typically has more severe symptoms and comes on suddenly, while a cold is milder and develops gradually."},
    {"question": "How do I know if I need to see a doctor?", "answer": "If you have persistent symptoms, severe pain, or any concerns about your health, it's best to consult a doctor."}

]
# Combine questions and answers
all_items = common_questions + faqs

# Initialize Qdrant client
qdrant_client = QdrantClient(
    url="https://d592a8e8-bf4f-42c2-8733-49bfdb35c394.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="d9oXnGCA1wBjcjUF977pT7rjAyBKh3-zgxOO2g95CQafCM9LAvf6VA",
)

collection_name = "health_faqs"

# Check if the collection exists
collections_response = qdrant_client.get_collections()
collections = collections_response.collections

# Define the correct vector parameters
vector_params = models.VectorParams(
    size=384,  # Dimension of the vectors (output of the sentence transformer)
    distance=models.Distance.COSINE  # Distance metric
)

# Create the collection if it doesn't exist
if collection_name not in [col.name for col in collections]:
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=vector_params,
    )
    print(f"Collection '{collection_name}' created.")

# Generate embeddings for the questions
embeddings = model.encode([item["question"] for item in all_items])

# Prepare points for upload with both question and answer in the payload
points = [
    {
        "id": idx,
        "vector": embedding.tolist(),  # Use the generated embedding as vector
        "payload": {
            "question": item["question"],
            "answer": item["answer"]
        }
    }
    for idx, (item, embedding) in enumerate(zip(all_items, embeddings))
]

# Attempt to upload data
try:
    response = qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )
    print("Upload response:", response)
except Exception as e:
    print("Error during upload:", e)

Collection 'health_faqs' created.
Upload response: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [ ]:
# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

collection_name = "health_faqs"

# Assuming `query_embedding` is your query vector
query_embedding = model.encode("What are the symptoms of diabetes?")  # Example query
query_embedding = query_embedding.flatten().tolist()  # Flatten the array and convert to list

# Perform the search
search_results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_embedding,
    limit=1  # Adjust as needed
)

# Print the results
for result in search_results:
    print(result)


id=0 version=0 score=1.0 payload={'question': 'What are the symptoms of diabetes?', 'answer': 'Common symptoms include increased thirst, frequent urination, and fatigue.'} vector=None shard_key=None order_value=None


In [ ]:
cohere_client = cohere.Client('VCNuTSP7TApcHO6k7xwpiyED7AjDJRRRgK9ASpR7')

def retrieve_documents(query, top_k=5):
  query_embedding = model.encode([query])[0].tolist()
  results = qdrant_client.search(
      collection_name=collection_name,
      query_vector=query_embedding,
      limit=top_k
  )
  return results

def generate_response(query, retrieved_docs):
  context = " ".join([result.payload["answer"] for result in retrieved_docs])
  response = cohere_client.generate(
      model = 'command-xlarge-nightly',
      prompt = f"Answer the question based on the context below. If the question cannot be answered using the information provided answer with 'I don't know, ask related to healthcare'.\n\nContext: {context}\n\n---\n\nQuestion: {query}\nAnswer:",
      max_tokens = 100,
      temperature = 0.5
  )
  return response.generations[0].text.strip()


def chatbot_response(user_input):
  retrieved_docs = retrieve_documents(user_input)
  response = generate_response(user_input, retrieved_docs)
  return response


while True:
  user_input = input("Ask a question: ")
  if user_input.lower() in ['exit', 'quit']:
    break
  response = chatbot_response(user_input)
  print("Chatbot:", response)

Ask a question: what is laptop
[ScoredPoint(id=15, version=0, score=0.17004906, payload={'question': 'What is hypertension?', 'answer': 'Hypertension, or high blood pressure, is a condition that can lead to serious health problems if not managed.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=30, version=0, score=0.14833382, payload={'question': 'What is sleep apnea?', 'answer': 'Sleep apnea is a condition where breathing repeatedly stops and starts during sleep.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=8, version=0, score=0.12750536, payload={'question': 'What is a balanced diet?', 'answer': 'A balanced diet includes a variety of foods: fruits, vegetables, whole grains, protein, and healthy fats.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=75, version=0, score=0.123946026, payload={'question': 'What is a health savings account (HSA)?', 'answer': 'An HSA is a tax-advantaged account that can be used to pay for eligible me